# Construção da base de notícias

In [3]:
import warnings
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
import unidecode
import datetime as dt
nltk.download('rslp')
nltk.download('punkt') 
from ipynb.fs.full.buscador_google_noticias import busca_noticias_google_news
from ipynb.fs.full.noticias_processamento_texto import remove_acentos, remove_termos_comuns, aplica_stemming_texto, remove_palavras_texto, conta_termos_esg, classifica_texto, classifica_textos_coletados, filtra_noticias_nao_relacionadas, filtra_noticias_sem_classificacao, conta_mencoes_empresas, filtra_citacao_relevante, remove_nome_composto

warnings.filterwarnings('ignore')
arquivo_termos_esg = 'palavras_chave_esg.xlsx'
base_noticias = 'base_noticias.xlsx'


'''
Busca as noticias do google 
'''
def busca_noticias_google_periodo(termo, atualiza=False, ultima_data=None):
    
    if not atualiza:
        df = busca_noticias_google_news(termo, '')  # busca sem data

        hoje = dt.datetime.now()
        for i in range(1, 15):   # busca 14 periodos de 6 meses
            dt_defasada = hoje - dt.timedelta(i * 180)

            dfNovaPesquisa = busca_noticias_google_news(termo, dt_defasada.strftime("%Y-%m-%d"))

            if len(dfNovaPesquisa) > 0:
                df = pd.concat([df, dfNovaPesquisa ])
    else:
        df = busca_noticias_google_news(palavras=termo, data_limite=ultima_data.strftime("%Y-%m-%d"), after=True)
          
            
    if len(df) > 0:

        df  = filtra_noticias_sites_especificos(df)

        df['fonte'] = df['fonte'].apply(lambda x : x['title'])

        df = df.drop_duplicates().reset_index(drop=True)
    
    return df



'''
Busca notícias de uma empresa relacionadas ao tema ESG
'''
def busca_noticias_google_esg(empresa_pesquisada, atualiza=False, ultima_data=None):
    dfTermos = pd.read_excel(arquivo_termos_esg)
    
    empresa_pesquisada_aju = empresa_pesquisada.replace(' ', '+')

    dfESG = busca_noticias_google_periodo(empresa_pesquisada_aju + '+ESG', atualiza, ultima_data)

    for letra in 'ESG':
        q = remove_acentos(empresa_pesquisada_aju + '+('+ '|'.join(''+ dfTermos[letra][dfTermos[letra].notnull()] +'').replace(' ', '+') + ')')
        dfLetra = busca_noticias_google_periodo(q, atualiza, ultima_data)
        
        if len(dfLetra) > 0:
            dfESG = pd.concat([dfESG,dfLetra]).drop_duplicates().reset_index(drop=True)
            
    if len(dfESG) > 0:
        dfESG = dfESG.sort_values(by='data_publicacao')  #ordena

        dfESG['empresa'] = remove_acentos(empresa_pesquisada.lower())
    
    return dfESG

'''
Filtra determinados sites
'''
def filtra_noticias_sites_especificos(noticias):
    df = noticias
    df = df[df['fonte'].str['href'].str.contains('xpi.')==False]
    df = df[df['fonte'].str['href'].str.contains('estrategiaconcursos')==False]
    df = df[df['fonte'].str['href'].str.contains('conjur')==False]
    df = df[df['fonte'].str['href'].str.contains('gov.br')==False]
    df = df[df['fonte'].str['href'].str.contains('portogente')==False]
    df = df[df['fonte'].str['href'].str.contains('stj.')==False]
    df = df[df['fonte'].str['href'].str.contains('pcb.')==False]

    return df

'''
Faz o scrapping de cada noticias utilizando biblioteca 
https://newspaper.readthedocs.io/en/latest/
'''
def recupera_noticias_completas(noticias):
    import pandas as pd
    from newspaper import Article 
    df = noticias
    textos = []

    for i in range(0, len(df)):
        url = df.iloc[i]['url']
        article = Article(url, language='pt')
        try:
            article.download()
            article.parse()
            textos.append(article.text)       
        except:
            textos.append('')     
    
    df['texto_completo'] = textos
    
    df = df[ (df['texto_completo'] != '')]
    
    return df




'''
Obtem relação de empresas listadas na B3
'''
def recupera_lista_empresas_B3():
    df = pd.read_html('https://www.dadosdemercado.com.br/bolsa/acoes')[0].loc[:, ['Código', 'Nome']].sort_values(by='Nome')

    df = pd.merge(left=df, right=pd.read_excel('EscoreB3.xlsx'), on='Código', how='left')
    
    df['Nome'] = df['Nome'].replace('AMER3', 'Lojas Americanas')
    df['Nome'] = df['Nome'].replace('VBBR3', 'VIBRA ENERGIA S.A.')
    df['Nome'] = df['Nome'].replace('RaiaDrogasil', 'Raia Drogasil')
    
    df.to_excel('lista_empresas.xlsx')
    

    
    return df[df['Nome'] != 'B3']

    

'''
 Gera um arquivo em excel com todas as empresas da B3 processadas
'''
def processa_base_noticias():
    
    dfEmpresasListadas = recupera_lista_empresas_B3()
    
    # seleciona apenas do ranking ISE
    dfEmpresasListadas = dfEmpresasListadas[~ pd.isnull(dfEmpresasListadas.RANKING) ]

    list_empresas = list(set(list(dfEmpresasListadas['Nome'])))

    
    list_empresas = [remove_acentos(i.lower()) for i in list_empresas]
    
    df_atual = pd.read_excel(base_noticias)
    
    for empresa in list_empresas:
        
        if len(df_atual[df_atual['empresa'] == empresa]) == 0:  # inclusao
        
            print ('empresa: ' + empresa + ' - ' + str(dt.datetime.now()))
            df = processa_busca_empresa(empresa, dfEmpresasListadas)
            
            if len(df) > 0:
                print('Adicionadas ' + str(len(df)) + ' notícias para ' + empresa  )
                df = pd.concat([df_atual,df]).drop_duplicates()
                df_atual = df

        df_atual.to_excel(base_noticias, index=False)
            
    return df_atual


'''
Processa as noticias de uma empresa de acordo com o processo descrito neste trabalho
'''
def processa_busca_empresa(empresa, dfEmpresasListadas):
    df = busca_noticias_google_esg(empresa)
    df = recupera_noticias_completas(df)
    if len(df) > 0:
        df = filtra_noticias_nao_relacionadas(df, empresa)
        df = filtra_citacao_relevante(df, empresa, dfEmpresasListadas )
        df = classifica_textos_coletados(df)
        df = filtra_noticias_sem_classificacao(df, empresa)
    return df    



[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
processa_base_noticias()

empresa: rumo - 2023-03-13 08:49:16.896045
Adicionadas 249 notícias para rumo
empresa: brf - 2023-03-13 09:43:34.535639
Adicionadas 229 notícias para brf
empresa: odontoprev - 2023-03-13 10:28:39.395585
Adicionadas 21 notícias para odontoprev
empresa: hypera - 2023-03-13 10:41:29.307342
Adicionadas 47 notícias para hypera
empresa: locaweb - 2023-03-13 11:01:54.378377
Adicionadas 66 notícias para locaweb
empresa: banco btg pactual - 2023-03-13 11:19:24.770401
Adicionadas 266 notícias para banco btg pactual
empresa: grupo pao de acucar - 2023-03-13 12:04:57.020842
Adicionadas 128 notícias para grupo pao de acucar
empresa: suzano - 2023-03-13 12:43:36.419622
Adicionadas 250 notícias para suzano
empresa: banco do brasil - 2023-03-13 13:39:18.988730
Adicionadas 440 notícias para banco do brasil
empresa: assai - 2023-03-13 14:37:38.584617
Adicionadas 145 notícias para assai
empresa: duratex - 2023-03-13 15:13:07.113058
Adicionadas 37 notícias para duratex
empresa: porto seguro - 2023-03-13 1

,titulo,data_publicacao,url,fonte,empresa,texto_completo,citacoes_empresa,demais_citacoes,relevante,classificacao
0,GetNinjas planeja expansão internacional - Épo...,2018-06-19 07:00:00,https://news.google.com/rss/articles/CBMiZWh0d...,Época NEGÓCIOS,getninjas,"Eduardo L’Hotellier, fundador da GetNinjas. Ex...",7,0,1,G
1,Saint-Gobain e GetNinjas se unem para expandir...,2018-09-21 07:00:00,https://news.google.com/rss/articles/CBMif2h0d...,Época NEGÓCIOS,getninjas,Como ter certeza da confiabilidade do profissi...,5,2,1,G
2,Entrevista: Telhanorte inaugura novo segmento ...,2019-10-22 07:00:00,https://news.google.com/rss/articles/CBMidWh0d...,Mercado e Consumo,getninjas,De acordo com dados da Associação Brasileira d...,1,1,1,S
3,Saiba quanto custa o processo de formação de u...,2019-11-06 08:00:00,https://news.google.com/rss/articles/CBMiX2h0d...,Forbes Brasil,getninjas,Divulgação Acessibilidade L L A- A+ ◐\n\n\n\n\...,1,1,1,S
4,"Após sucesso, telhanorteJá! chega ao bairro de...",2019-11-29 08:00:00,https://news.google.com/rss/articles/CBMiZmh0d...,Mercado e Consumo,getninjas,A Telhanorte abriu nesta semana a segunda unid...,1,1,1,S
...,...,...,...,...,...,...,...,...,...,...
0,Grupo Soma celebra resultado de R$ 6 bilhões e...,2023-02-18 08:00:00,https://news.google.com/rss/articles/CBMib2h0d...,Forbes Brasil,grupo soma,Foto: Divulgação/ SOMA Acessibilidade L L A- A...,7,4,1,E
659,"Prioridades políticas para o G20: Uma Terra, U...",2023-02-24 08:00:00,https://news.google.com/rss/articles/CBMiYmh0d...,A Referência,grupo soma,Este artigo foi publicado originalmente em ing...,1,0,1,E
902,Madeireiras com selo verde recebem multas mili...,2023-03-01 08:00:00,https://news.google.com/rss/articles/CBMibWh0d...,Poder360,grupo soma,Investigação internacional mostra que certific...,2,0,1,E
750,Perfil mais conservador do Congresso pode impa...,2023-03-08 13:06:55,https://news.google.com/rss/articles/CBMic2h0d...,Tudo Rondônia,grupo soma,Parlamentares da nova legislatura têm um posic...,1,0,1,E


In [ ]:
#empresa = 'b3'
#dfEmpresasListadas = recupera_lista_empresas_B3()
#df1 = busca_noticias_google_esg('ambipar')
#df2 = recupera_noticias_completas(df1)
#df3 = filtra_noticias_nao_relacionadas(df2, empresa)
#df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
#df5 = classifica_textos_coletados(df4)
#df6 = filtra_noticias_sem_classificacao(df5, empresa)